<a href="https://colab.research.google.com/github/lokeshtalamala1/Deep_Learning_Assignment_1/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#initializing my project and entity in wandb using its atributes
#Deep Learning Assignment 1
#CS24M023

!pip install wandb tensorflow
import wandb
wandb.init(project='Deep_Learning_Assignment_1', entity='cs24m023-indian-institute-of-technology-madras')


In [4]:
#import numpy ,fashion_mnist and load data

from keras.datasets import fashion_mnist
import numpy as np

((x_train,y_train),(x_test,y_test)) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
import matplotlib.pyplot as plt

In [6]:
#example image from each class from given data

#QUESTION 1

images = []
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for class_label in range(len(classes)):
    index = next(i for i, y in enumerate(y_train) if y == class_label)
    images.append(wandb.Image(x_train[index], caption=classes[class_label]))

wandb.log({"example image from each class": images})
plt.show()
wandb.finish()

In [8]:
#creating a class with a layer as an object and parametes to the layer are :
    #input dimensions
    #activation_function
    #optimizer_function
    #weight initialization #QUESTION 2 AND QUESTION 3

import numpy as np

class SingleLayer:
    def __init__(self, input_dim, num_nodes, activation='softmax', optimizer='gradient_descent', weight_type='random'):
        self.input_dim = input_dim
        self.num_nodes = num_nodes
        self.activation = activation
        self.optimizer_name = optimizer
        self.weights, self.bias = self.initialize_weights(weight_type)

        # Momentum and velocity terms
        self.momentum_w = np.zeros((num_nodes, input_dim))
        self.momentum_b = np.zeros((num_nodes, 1))
        self.velocity_w = np.zeros((num_nodes, input_dim))
        self.velocity_b = np.zeros((num_nodes, 1))

        # Function mappings
        self.activation_func, self.activation_deriv = self.get_activation_function(activation)
        self.optimizer_func = self.get_optimizer_function(optimizer)

    def initialize_weights(self, weight_type):
        np.random.seed(1)
        if weight_type == 'random':
            weights = np.random.normal(0, 0.5, (self.num_nodes, self.input_dim))
        else:  # Xavier Initialization
            limit = np.sqrt(1 / self.input_dim)
            weights = np.random.uniform(-limit, limit, (self.num_nodes, self.input_dim))
        bias = np.ones((self.num_nodes, 1))
        return weights, bias

    def get_activation_function(self, activation):
        activations = {
            'sigmoid': (self.sigmoid, self.sigmoid_derivative),
            'relu': (self.relu, self.relu_derivative),
            'tanh': (self.tanh, self.tanh_derivative),
            'softmax': (self.softmax, self.softmax_derivative)
        }
        return activations.get(activation, activations['softmax'])

    def get_optimizer_function(self, optimizer):
        optimizers = {
            'gradient_descent': self.gradient_descent,
            'momentum_gradient_descent': self.momentum_gradient_descent,
            'rmsprop': self.rmsprop,
            'adam': self.adam,
            'stochastic_gradient_descent': self.stochastic_gradient_descent,
            'nadam': self.nadam,
            'nesterov': self.nesterov
        }
        return optimizers.get(optimizer, self.gradient_descent)

    # Activation Functions
    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-np.clip(Z, -500, 500)))

    def sigmoid_derivative(self, A):
        return A * (1 - A)

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_derivative(self, A):
        return (A > 0).astype(float)

    def tanh(self, Z):
        return np.tanh(Z)

    def tanh_derivative(self, A):
        return 1 - A ** 2

    def softmax(self, Z):
        expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return expZ / np.sum(expZ, axis=0, keepdims=True)

    def softmax_derivative(self, A):
        return A * (1 - A)

    # Forward Propagation
    def forward_propagate(self, A):
        self.prev_A = A
        Z = np.dot(self.weights, A) + self.bias
        self.prev_Z = Z
        return self.activation_func(Z)

    # Backward Propagation
    def backward_propagate(self, dA):
        m = self.prev_A.shape[1]
        dZ = self.activation_deriv(self.prev_Z) * dA
        self.dW = np.dot(dZ, self.prev_A.T) / m
        self.dB = np.sum(dZ, axis=1, keepdims=True) / m
        return np.dot(self.weights.T, dZ)

     # Optimizers
    def gradient_descent(self, learn_rate, l2_lambda=0, batch_size=32):
        self.weights -= learn_rate * (self.dW + l2_lambda / batch_size * self.weights)
        self.bias -= learn_rate * self.dB

    def momentum_gradient_descent(self, learn_rate, beta=0.9, l2_lambda=0, batch_size=32):
        self.momentum_w = beta * self.momentum_w + learn_rate * self.dW
        self.momentum_b = beta * self.momentum_b + learn_rate * self.dB
        self.weights -= self.momentum_w + l2_lambda / batch_size * self.weights
        self.bias -= self.momentum_b

    def rmsprop(self, learn_rate, beta=0.9, epsilon=1e-8, l2_lambda=0, batch_size=32):
        self.velocity_w = beta * self.velocity_w + (1 - beta) * np.square(self.dW)
        self.velocity_b = beta * self.velocity_b + (1 - beta) * np.square(self.dB)
        self.weights -= learn_rate * self.dW / (np.sqrt(self.velocity_w) + epsilon) + l2_lambda / batch_size * self.weights
        self.bias -= learn_rate * self.dB / (np.sqrt(self.velocity_b) + epsilon)

    def adam(self, learn_rate, beta1=0.9, beta2=0.999, epsilon=1e-8, t=1):
        self.momentum_w = beta1 * self.momentum_w + (1 - beta1) * self.dW
        self.momentum_b = beta1 * self.momentum_b + (1 - beta1) * self.dB
        self.velocity_w = beta2 * self.velocity_w + (1 - beta2) * np.square(self.dW)
        self.velocity_b = beta2 * self.velocity_b + (1 - beta2) * np.square(self.dB)

        corrected_mw = self.momentum_w / (1 - beta1 ** t)
        corrected_mb = self.momentum_b / (1 - beta1 ** t)
        corrected_vw = self.velocity_w / (1 - beta2 ** t)
        corrected_vb = self.velocity_b / (1 - beta2 ** t)

        self.weights -= learn_rate * corrected_mw / (np.sqrt(corrected_vw) + epsilon)
        self.bias -= learn_rate * corrected_mb / (np.sqrt(corrected_vb) + epsilon)

    def nesterov(self, learn_rate, beta=0.9):
        temp_w = self.weights - beta * self.momentum_w
        temp_b = self.bias - beta * self.momentum_b
        self.weights -= learn_rate * self.dW + beta * temp_w
        self.bias -= learn_rate * self.dB + beta * temp_b

    def nadam(self, learn_rate, beta1=0.9, beta2=0.999, epsilon=1e-8, t=1):
        mw_hat = (beta1 * self.momentum_w) / (1 - beta1 ** t) + self.dW
        mb_hat = (beta1 * self.momentum_b) / (1 - beta1 ** t) + self.dB
        vw_hat = (beta2 * self.velocity_w) / (1 - beta2 ** t)
        vb_hat = (beta2 * self.velocity_b) / (1 - beta2 ** t)

        self.weights -= learn_rate * mw_hat / (np.sqrt(vw_hat) + epsilon)
        self.bias -= learn_rate * mb_hat / (np.sqrt(vb_hat) + epsilon)

    # Prediction
    def predict(self, A):
        return self.forward_propagate(A)

In [9]:
# QUESTION 2 AND QUESTION 3 FORWARD PROPAGAE AND BACKWARD PROPAGATE
#feedforwardneuralnetwork for calculating all the layers over the model and loss functions are calculated
#with input parameters optimizer,activation function weight initialization
#number of epochs and size of each layer ,learnig rate theeta
from sklearn.model_selection import train_test_split

class FeedForwardNeuralNetwork:
    def __init__(self, layers_size, epochs=5, learn_rate=0.001, l2_lambda=0, optimizer='gradient_descent',
                 activation='sigmoid', weight_type='random', loss='cross_entropy'):
        self.layers = []
        self.layers_size = layers_size
        self.epochs = epochs
        self.learn_rate = learn_rate
        self.optimizer = optimizer
        self.activation = activation
        self.weight_type = weight_type
        self.l2_lambda = l2_lambda
        self.loss = loss

        # Assigning loss functions
        loss_functions = {
            'mean_square': (self.mean_square, self.mean_square_grad),
            'cross_entropy': (self.cross_entropy, self.cross_entropy_grad)
        }
        self.losscomputation, self.lossBackwardpass = loss_functions.get(loss, (None, None))

        if self.losscomputation is None:
            raise ValueError("Invalid loss function selected.")

    def addingLayer(self, input_dim=None, num_nodes=1, activation='', weight_type='random'):
        """Add a new layer to the neural network."""
        if not self.layers and input_dim is None:
            raise ValueError('Invalid input dimensions for the first layer.')
        input_dim = input_dim if input_dim is not None else self.layers[-1].outputDimension()
        self.layers.append(SingleLayer(input_dim, num_nodes, activation, optimizer=self.optimizer, weight_type=weight_type))

    # Loss functions and gradients
    def mean_square(self, Y, A):
        return np.mean(np.square(Y - A))

    def mean_square_grad(self, Y, A):
        return -2 * (Y - A)

    def cross_entropy(self, Y, A):
        return -np.mean(np.sum(Y * np.log(A + 1e-9), axis=0))

    def cross_entropy_grad(self, Y, A):
        return A - Y

    def cost(self, Y, A):
        """Computes the loss for given true and predicted values."""
        return self.losscomputation(Y, A)

    def forward_propagate(self, X):
        """Pass input through all layers of the network."""
        for layer in self.layers:
            X = layer.forward_propagate(X)
        return X

    def backward_propagate(self, Y, A):
        """Backpropagation step."""
        dA = self.lossBackwardpass(Y, A)
        for layer in reversed(self.layers):
            dA = layer.backward_propagate(dA)

    def update_weights(self, batch_size, t=0):
        """Update the weights using the chosen optimizer."""
        for layer in self.layers:
            layer.optimizer(self.learn_rate, l2_lambda=self.l2_lambda, batch_size=batch_size, t=t)

    def fit(self, x_train, y_train, x_test, y_test, batch_size=32):
        """Train the neural network using the provided dataset."""
        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, test_size=0.1, random_state=10)

        if self.activation == 'relu':
            self.weight_type = 'xavier'

        # Add layers to the model
        for i in range(1, len(self.layers_size) - 1):
            self.addingLayer(input_dim=self.layers_size[i - 1], num_nodes=self.layers_size[i], activation=self.activation, weight_type=self.weight_type)

        # Output layer (softmax)
        self.addingLayer(input_dim=self.layers_size[-2], num_nodes=self.layers_size[-1], activation='softmax', weight_type=self.weight_type)

        # One-hot encoding
        y_train_encoded = np.eye(len(set(y_train)))[y_train].T

        for epoch in range(self.epochs):
            # Adjust learning rate if using specific optimizers
            if self.activation == 'relu' and self.optimizer in {'momentum_gradient_descent', 'nesterov', 'rmsprop'}:
                self.learn_rate /= 15

            for i in range(0, x_train.shape[0], batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train_encoded[:, i:i + batch_size]

                x_batch = x_batch.reshape(x_batch.shape[0], -1).T
                x_batch = (x_batch - x_batch.min()) / (x_batch.max() - x_batch.min())

                A = self.forward_propagate(x_batch)
                self.backward_propagate(y_batch, A)
                self.update_weights(batch_size=batch_size, t=epoch + 1)

            # Evaluate after each epoch
            val_loss, val_acc, _ = self.predict(x_val, y_val)
            test_loss, test_acc, y_pred = self.predict(x_test, y_test)

            print(f"After {epoch + 1} iterations:")
            print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")
            print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

        return y_pred

    def predict(self, x, y):
        """Predict outputs for the given input data."""
        x = x.reshape(x.shape[0], -1).T
        y_encoded = np.eye(len(set(y)))[y].T

        for layer in self.layers:
            x = layer.predict(x)

        cross_entropy_loss = -np.mean(np.sum(y_encoded * np.log(x + 1e-9), axis=0))
        y_pred = np.argmax(x, axis=0)
        accuracy = np.mean(y == y_pred)

        return cross_entropy_loss, accuracy, y_pred


In [10]:
import wandb

sweep_config = {
    'name': "my-sweep",
    'method': 'bayes',
    'metric': {'name': 'accuracy', 'goal': 'maximize'},
    'parameters': {
        'epochs': {'values': [5, 10]},
        'hidden_layer': {'values': [3, 4, 5]},
        'hidden_size': {'values': [32, 64, 128]},
        'weight_decay': {'values': [0, 0.0005, 0.5]},
        'learn_rate': {'values': [1e-3, 1e-4]},
        'optimizer': {
            'values': ['momentum_gradient_descent', 'nesterov', 'rmsprop',
                       'adam', 'nadam', 'stochastic_gradient_descent']
        },
        'batch_size': {'values': [16, 32, 64]},
        'weight_initial': {'values': ['random', 'xavier']},
        'activation': {'values': ['sigmoid', 'tanh', 'relu']}
    }
}

# Initializing sweep with WandB
sweep_id = wandb.sweep(sweep_config,
                       entity="cs24m023-indian-institute-of-technology-madras",
                       project="Deep_Learning_Assignment_1")


Create sweep with ID: cgk2fp87
Sweep URL: https://wandb.ai/cs24m023-indian-institute-of-technology-madras/Deep_Learning_Assignment_1/sweeps/cgk2fp87
